# HashCode 2020 Qualifier
Problem : https://storage.googleapis.com/coding-competitions.appspot.com/HC/2020/hashcode_2020_online_qualification_round.pdf

Data : https://storage.googleapis.com/coding-competitions.appspot.com/HC/2020/qualification_round_2020.in.zip


In [1]:
def parse(data):
    '''
    Parse the file to create a meta dictionary.
    
    Input: A list with each item representing a line from input file
    returns: A dictionary of parsed values
    '''
    info = dict()
    line1 = data[0].split()
    no_of_books, no_of_lib, no_of_days = int(line1[0]), int(line1[1]), int(line1[2])
    book_scores = [int(i) for i in data[1].split()]
    info['no_of_books'] = no_of_books
    info['no_of_lib'] = no_of_lib
    info['no_of_days'] = no_of_days
    info['book_scores'] = book_scores
    
    data = data[2:]
    for i in range(no_of_lib):
        data_lib = data[i*2 : i*2+2]
        info[f'lib_id_{i}'] = __parse_lib__(data_lib, i)

    return info
        
def __parse_lib__(data, id_):
    lib = dict()
    t = data[0].split()
    lib['id'] = id_
    lib['no_of_books'], lib['signup_time'], lib['can_ship'] = int(t[0]), int(t[1]), int(t[2])
    lib['books'] = [int(i) for i in data[1].split()]
    return lib

In [85]:
with open('files/e_so_many_books.txt', 'rb') as f:
    lines = f.readlines()

data = []
for line in lines:
    data.append(line.decode('utf-8').strip())
    
info = parse(data)

In [86]:
# Sort the libraries by their signup time in descending order

# Figure out the order in which the books are to be shipped out - constraint is to have the scores maximized
    # you could sort the books in the order of the scores
    # mind you that other libraries may have sent the same book ids, so you'll have to send out the ones that havent yet been sent out, but have the highest scores

In [87]:
def sort_lib_by_signup_time(invert=False):
    sorted_list_of_lib = []
    for key in info.keys():
        if 'lib_id' in key:
            sorted_list_of_lib.append(info[key])
    sorted_list_of_lib = sorted(sorted_list_of_lib, key=lambda x:x['signup_time'], reverse=invert)
    return sorted_list_of_lib

def get_scores_dict():
    keys = []
    values = []
    for i, j in enumerate(info['book_scores']):
        keys.append(i)
        values.append(j)
    return dict(zip(keys, values))

# Constraint maximization strategy

For each library ->

```no_of_scan_days = no_of_days - signup_time``` (Signup time of all the other libraries already considered)

```total_no_books_from_a_lib = no_of_scan_days * can_ship```

Now it becomes a game of combination
Get all combination of ```total_no_books_from_a_lib``` from each library
Choose the combination that gives the best score

How do you consider which libraries to choose to sign up, as picking up a library at the expense of dropping others may not maximize the scores?
Knapsack problem like algorithm, where you consider a library for inclusion if remaining_days-its_signup_time 

# Reference code Knapsack
def knapsack(values, weights, rem_capacity, n):
    if rem_capacity == 0 or n == 0:
        return 0
    else:
        if rem_capacity >= weights[n-1]:
            output = max((values[n-1]+knapsack(values, weights, rem_capacity-weights[n-1], n-1)), 
                         knapsack(values, weights, rem_capacity, n-1))
        else:
            output = knapsack(values, weights, rem_capacity, n-1)
        
        return output


weights =  [1, 3, 5, 7]
values = [100, 80, 40, 30]
capacity = 8
knapsack(values, weights, capacity, len(weights))

capacity = 11
weights = [1, 2, 5, 6, 7]
values = [1, 6, 18, 22, 28]
knapsack(values, weights, capacity, len(weights))

### Using Naive Scoring strategy - use descending order of scores from each library in the order of their signup times and sum the scores

In [95]:
# sorted_list_of_lib = sort_lib_by_signup_time()
days = info['no_of_days']
scores_dict = get_scores_dict()

# Debugging purposes
books_sent = set()

lib_counter = 0
for lib in sorted_list_of_lib:
    while days - lib['signup_time'] > 0:
        days -= lib['signup_time']
#         print(f"Lib {lib['id']}, Days Remaining : {days}")
        can_ship = lib['can_ship']
        scan_days = lib['no_of_books']/can_ship # TODO : Account for fractional values
        if days >= scan_days:
            # All books in this library can be scanned and order doesnt matter
            print(str(lib['id'])+ ' '+str(len(lib['books'])))
            #print(*lib['books'])
            
            # For score tracking
            for book in lib['books']:
                books_sent.add(book)
            # Break out after processing a library TODO : Revisit this piece of junk

            lib_counter += 1
            break            
        else:
            # Send the books by the order of the scores - higher ones first
            book_scores_tuple = [(book, scores_dict[book]) for book in lib['books']]
            book_scores_tuple = sorted(book_scores_tuple, key=lambda x:x[1], reverse=True)
#             print(f'days : {days}, can_ship: {can_ship}')
            no_of_books_sent_for_scanning = days*can_ship
            if lib['no_of_books'] < no_of_books_sent_for_scanning:
                no_of_books_sent_for_scanning = lib['no_of_books']
            print(str(lib['id'])+ ' '+str(no_of_books_sent_for_scanning))
            for i in range(no_of_books_sent_for_scanning):
                books_sent.add(book_scores_tuple[i][0])
            #print(*[str(book_scores_tuple[i][0]) for i in range(no_of_books_sent_for_scanning)])

            lib_counter += 1
            break            
#         print(f'scan_days : {scan_days}')

198 4
13 2
2 42
341 126
233 75
221 21
175 56
49 30
70 7
766 220
595 87
566 270
316 226
31 264
127 246
446 228
409 222
628 24
261 155
470 180
666 174
865 164
41 73
434 144
284 130
353 122
742 102
996 96
861 84
677 70
226 56
697 38
402 24
374 10
837 4
388 2


In [96]:
sum([scores_dict[id] for id in books_sent])

636075

In [98]:
lib_counter

36

# Method 2 - sorting the libraries by their best scores

In [93]:
def sort_lib_by_best_scores(invert=True):
    sorted_list_of_lib = []
    for key in info.keys():
        if 'lib_id' in key:
            sorted_list_of_lib.append(info[key])
            # Uses a heuristic that the score should be high, with a lower number of books and the number that can ship per day should be high.
            info[key]['score'] = sum([scores_dict[i] for i in info[key]['books']])
            info[key]['score'] = (info[key]['score']*info[key]['can_ship'])/len(info[key]['books'])
    sorted_list_of_lib = sorted(sorted_list_of_lib, key=lambda x:x['score'], reverse=invert)
    return sorted_list_of_lib

In [94]:
sorted_list_of_lib = sort_lib_by_best_scores()

In [66]:
sorted_list_of_lib[0]['score']

9999800.0